In [2]:
# !pip install -U transformers accelerate datasets bertviz
!pip install -q -U datasets

In [18]:
import pprint

from datasets import load_dataset

# from transformers import BartForConditionalGeneration

from transformers import BartForQuestionAnswering

from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import TrainingArguments, Trainer

# Import Data

In [4]:
ds = load_dataset("WorkInTheDark/FairytaleQA")

In [5]:
pprint.pprint(ds['train'][1])

{'answer1': 'kind and just .',
 'answer2': '',
 'attribute': 'character',
 'ex-or-im': 'explicit',
 'ex-or-im2': '',
 'local-or-sum': 'local',
 'question': 'what type of ruler was the king ?',
 'story_name': 'three-dogs',
 'story_section': 'once upon a time there was a king who went forth into the '
                  'world and fetched back a beautiful queen . and after they '
                  'had been married a while god gave them a little daughter . '
                  'then there was great rejoicing in the city and throughout '
                  'the country , for the people wished their king all that was '
                  'good , since he was kind and just . while the child lay in '
                  'its cradle , a strange - looking old woman entered the room '
                  ', and no one knew who she was nor whence she came . the old '
                  'woman spoke a verse over the child , and said that she must '
                  'not be allowed out under the open sky 

In [6]:
train = ds['train'].shuffle()
val = ds['validation'].shuffle()
test = ds['test'].shuffle()

#Model & Tokenizer

In [20]:
bartmodel = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
barttokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [23]:
print(bartmodel)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

#Data Preperation

In [24]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "
context = " Context: "

# Define the preprocessing function

def preprocess_function(data, tokenizer):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + question + context for question,context in zip(data["question"],data['story_section'])]
   model_inputs = tokenizer(text_target=inputs,
                      max_length=512,
                      truncation=True,
                      padding='max_length',
                            return_tensors='pt')

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=data["answer1"],
                      max_length=512,
                      truncation=True,
                      padding='max_length',
                      return_tensors='pt')

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [25]:
# Map the preprocessing function across our dataset
train_tokenized = train.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': barttokenizer})
val_tokenized = val.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': barttokenizer})
test_tokenized = test.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': barttokenizer})

train_tokenzied = {'input_ids': train_tokenized['input_ids'], 'labels': train_tokenized['labels']}
val_tokenzied = {'input_ids': val_tokenized['input_ids'], 'labels': val_tokenized['labels']}
test_tokenzied = {'input_ids': test_tokenized['input_ids'], 'labels': test_tokenized['labels']}

Map:   0%|          | 0/8548 [00:00<?, ? examples/s]

Map:   0%|          | 0/1025 [00:00<?, ? examples/s]

Map:   0%|          | 0/1007 [00:00<?, ? examples/s]

#One Example

In [14]:
train_tokenized[0].keys()

dict_keys(['story_name', 'story_section', 'question', 'answer1', 'answer2', 'local-or-sum', 'attribute', 'ex-or-im', 'ex-or-im2', 'input_ids', 'attention_mask', 'labels'])

In [ ]:
# # # data preperation

# from transformers import pipeline
# pipe_bart=pipeline('summarization',model='facebook/bart-large-cnn')

# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import torch
# samsum = load_dataset('knkarthick/samsum')
# model_ckpt='facebook/bart-large-cnn'
# tokenizer=AutoTokenizer.from_pretrained(model_ckpt)
# model=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

Device set to use cpu


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [ ]:
# print(samsum)
# print(samsum['train'][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})
{'id': '13818513', 'dialogue': "Amanda: I baked  cookies. Do you want some?\nJerry: Sure!\nAmanda: I'll bring you tomorrow :-)", 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}


In [ ]:
# # from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# # tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
# # model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')

# def get_feature(batch):
#     encodings = tokenizer(batch['dialogue'], text_target=batch['summary'], max_length=1024, truncation=True)
#     encodings = {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask'], 'labels': encodings['labels']}
#     return encodings

# samsum_pt = samsum.map(get_feature, batched=True)

# columns=['input_ids','labels','attention_mask']
# samsum_pt.set_format(type='torch',columns=columns)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
# import torch
# from transformers import pipeline

# pipeline = pipeline(
#     task="fill-mask",
#     model="facebook/bart-large",
#     torch_dtype=torch.float16,
#     device=0
# )
# pipeline("Plants create <mask> through a process known as photosynthesis.")

In [ ]:
# # # data preperation
# # from datasets import load_dataset
# from transformers import pipeline
# pipe_bart=pipeline('summarization',model='facebook/bart-large-cnn')
# # from datasets import load_dataset
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# import torch
# # samsum = load_dataset('samsum')
# model_ckpt='facebook/bart-large-cnn'
# tokenizer=AutoTokenizer.from_pretrained(model_ckpt)
# model=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

# # data preprocessing
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
# def get_feature(batch):
#     encodings = tokenizer(batch['dialogue'], text_target=batch['summary'], max_length=1024, truncation=True)
#     encodings = {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask'], 'labels': encodings['labels']}
#     return encodings
# samsum_pt = samsum.map(get_feature, batched=True)
# columns=['input_ids','labels','attention_mask']
# samsum_pt.set_format(type='torch',columns=columns)

# # model training
# from transformers import DataCollatorForSeq2Seq
# data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model)
# from transformers import TrainingArguments,Trainer
# training_args=TrainingArguments(
#     output_dir='bart_samsum',
#     num_train_epochs=1,
#     warmup_steps=500,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     weight_decay=0.01,
#     logging_steps=10,
#     evaluation_strategy='steps',
#     eval_steps=500,
#     save_steps=1e6,
#     gradient_accumulation_steps=16
# )
# trainer=Trainer(model=model,
#                 args=training_args,
#                 tokenizer=tokenizer,
#                 data_collator=data_collator,
#                 train_dataset=samsum_pt['train'],
#                 eval_dataset=samsum_pt['validation'])

# # evaluation
# eval_results = trainer.evaluate(samsum_pt['test'])
# print(f"Evaluation results: {eval_results}")
# dialogue = "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
# input_ids = tokenizer(dialogue, return_tensors='pt', truncation=True).input_ids
# output = model.generate(input_ids)
# summary = tokenizer.decode(output[0], skip_special_tokens=True)
# print(f"Summary: {summary}")